In [81]:
# This script examines the UCI sanctioned racing calendar to look at global changes in the number of events.
# Source is https://www.uci.org/road/calendar, all categories/classes selected
# Year could be extracted from the start date, but UCI 'seasons' differ from calendar years
# for Africa, Asia and Oceania circuits, so when outputting new calendars, add the year as _2XXX before .xlsx
# The script will add the year as a new column.

In [82]:
import pandas as pd
import glob
import os
import time
#Map the UCI Calendar excel output to the Continental circuits using the UCI's country names
continents = {'Americas': ['ANTIGUA AND BARBUDA', 'ARGENTINA', 'ARUBA', 'BELIZE', 'BERMUDA', 
                           'BOLIVARIAN REPUBLIC OF VENEZUELA', 'BOLIVIA', 'BRAZIL', 'CANADA',
                           'CHILE', 'COLOMBIA', 'CUBA', 'DOMINICAN REPUBLIC', 'ECUADOR', 
                           'EL SALVADOR', 'HONDURAS', 'PANAMA', 'PARAGUAY', 'PUERTO RICO', 
                           'SAINT VINCENT AND THE GRENADINES', 'TRINIDAD AND TOBAGO', 
                           'UNITED STATES OF AMERICA', 'URUGUAY','MEXICO', 'COSTA RICA','GUATEMALA'],
              'Africas' : ['ALBANIA', 'ALGERIA', 'ANGOLA', 'CAMEROON', 'CONGO', 'COTE D\'IVOIRE',
                           'EGYPT', 'ETHIOPIA', 'GUYANA', 'LIBYA', 'MALI', 'MAURITIUS', 'MOROCCO',
                           'NAMIBIA', 'RWANDA', 'SENEGAL', 'SOUTH AFRICA', 'SWAZILAND', 'TUNISIA', 
                           'UGANDA', 'ZIMBABWE', 'GABON','ERITREA', 'BURKINA FASO'],
              'Asia' : ['AZERBAIJAN', 'BAHRAIN', 'BRUNEI DARUSSALAM', 'CHINESE TAIPEI', 'GEORGIA',
                        'INDIA', 'ISLAMIC REPUBLIC OF IRAN', 'ISRAEL', 'KAZAKHSTAN', 'KOREA', 'KUWAIT', 
                        'KYRGYZSTAN', 'LEBANON', 'MALAYSIA', 'MONGOLIA', 'MYANMAR', 'OMAN', 'PHILIPPINES',
                        'RUSSIAN FEDERATION', 'SINGAPORE', 'SRI LANKA', 'SYRIAN ARAB REPUBLIC', 'THAILAND',
                        'TURKEY', 'UNITED ARAB EMIRATES', 'VIETNAM','HONG KONG, CHINA','QATAR', 'UZBEKISTAN',
                        'INDONESIA', 'JAPAN','PEOPLE\'S REPUBLIC OF CHINA'],
              'Europe' : ['AUSTRIA', 'BELARUS', 'BELGIUM', 'BOSNIA AND HERZEGOVINA', 'BULGARIA', 'CROATIA',
                          'CYPRUS', 'CZECH REPUBLIC', 'DENMARK', 'ESTONIA', 'FINLAND', 
                          'FORMER YUGOSLAV REPUBLIC OF MACEDONIA', 'FRANCE', 'GERMANY', 'GREAT BRITAIN', 
                          'GREECE', 'HUNGARY', 'ICELAND', 'IRELAND', 'ITALY', 'KOSOVO', 'LATVIA', 'LITHUANIA',
                          'LUXEMBOURG', 'MONTENEGRO', 'NETHERLANDS', 'NORWAY', 'POLAND', 'PORTUGAL', 
                          'REPUBLIC OF MOLDOVA', 'ROMANIA', 'SAN MARINO', 'SERBIA', 
                          'SLOVAKIA', 'SLOVENIA', 'SWEDEN', 'SWITZERLAND', 'UKRAINE','SPAIN'],
              'Oceania' : ['NEW ZEALAND','AUSTRALIA']}

In [83]:
#Pull in every excel sheet from the current folder that starts with Calendar and ends in xlsx. YMMV!

all_data = pd.DataFrame()
for f in glob.glob('Calendar*.xlsx'):
    df = pd.read_excel(f, header=1, encoding='UTF-8')
    year = os.path.basename(f).split('.')[0].split('_')[-1]
    df['Season'] = year
    all_data = all_data.append(df,ignore_index=True)

In [84]:
#Examine the head and length to make sure it's kosher
# all_data['Class'].unique()
# array(['2.2', 'CRT', '2.HC', '2.1', '1.2', '1.1', 'CN', '2.UWT', '1.UWT',
#        '2.Ncup', '1.HC', 'CC', '1.WWT', '1.Ncup', '1.2U', '2.2U', 'JR',
#        '2.WWT', 'JC', 'CM', 'JOJ', 'CDM', nan, 'JO', 'CPE', 'MNM', 'AU1',
#        '2.CH', '1.CH', 'GT2', 'GT1'], dtype=object)

# I suspect there are some classes have changed over the years. MP is ME, CPE was WT stage races during dispute
# with ASO (2.UWT), MNM = Monument and CPE were WT one-days during disputed years (1.UWT)
all_data['Category'].replace('MP', 'ME')
all_data['Class'].replace('CPE', '2.UWT')
all_data['Class'].replace(['MNM','AU1', '1.CH'], '1.UWT')

all_data['Class'].unique()

array(['2.2', 'CRT', '2.HC', '2.1', '1.2', '1.1', 'CN', '2.UWT', '1.UWT',
       '2.Ncup', '1.HC', 'CC', '1.WWT', '1.Ncup', '1.2U', '2.2U', 'JR',
       '2.WWT', 'JC', 'CM', 'JOJ', 'CDM', nan, 'JO', 'CPE', 'MNM', 'AU1',
       '2.CH', '1.CH', 'GT2', 'GT1'], dtype=object)

In [88]:
#Get rid of unwanted columns.
del all_data['EMail']
del all_data['WebSite']
del all_data['Calendar']
del all_data['Venue']
all_data.rename[{'Date From': 'Start_Date', 'Date To': 'End_Date'}]
all_data.head()

KeyError: 'EMail'

In [79]:
#Need to reformat the dates

all_data['Date To'] = all_data['Date To'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y')[0])
all_data.head()

,Date From,Date To,Name,Country,Category,Class,Season
0,24/10/2017,1,Vuelta a Colombia Femenina Oro y Paz,COLOMBIA,WE,2.2,2018
1,23/10/2017,0,Vuelta a Guatemala,GUATEMALA,ME,2.2,2018
2,04/11/2017,0,Subaru Australian Open Criterium,AUSTRALIA,ME,CRT,2018
3,04/11/2017,0,Le Tour De France Saitama Criterium,JAPAN,ME,CRT,2018
4,04/11/2017,0,Subaru Australian Open Criterium,AUSTRALIA,WE,CRT,2018


In [36]:
for x in ['CC', 'CM', 'CN']:
    row = (all_data.loc[all_data['Class'] == x]).sort_values(by='Season', ascending=0).iloc[0]
label = row.name
non_championships_df = all_data.drop(label)

non_championships_df.reset_index
#non_championships_df.head()
non_championships_df.dtypes

Date From    object
Date To      object
Name         object
Country      object
Category     object
Class        object
Season       object
dtype: object

In [9]:
#You can extract the year from the start date, but UCI 'seasons' differ from calendar years
# for Africa, Asia and Oceania circuits, so when outputting new calendars, add the year as _2XXX before .xlsx
# If you want you can just export the combined data

# all_data['Year'] = pd.DatetimeIndex(all_data['Date From']).year

# writer = pd.ExcelWriter('combined.xlsx')
# all_data.to_excel(writer,'Sheet1')

# writer.save()

In [47]:
all_data['Date From'] = all_data.iloc[:, 1]
# all_data['Date To'].astype('datetime64')


# all_data['Race Days'] = all_data['Date To'].astype('datetime64') - all_data['Date From'].astype('datetime64')
all_data.head()

AttributeError: 'Series' object has no attribute 'split'

In [21]:
#We need to make the dictionary of lists one big dictionary to add the Continents

cont_dict_converted = {k: oldk for oldk, oldv in continents.items() for k in oldv}

#Note there are some events that cross borders or Continental championships that have no Continent.
#Add some code here to handle these cases.

non_championships_df['Continent'] = non_championships_df['Country'].map(cont_dict_converted).fillna('Stateless')


In [34]:
#Let's get rid of the national, continental championships and Olympics to remove null categories
cats = ['ME', 'WE']
non_championships_df.Category.isin(cats)
elite_non_championships = non_championships_df[non_championships_df.Category.isin(cats)]
# group_cont = elite_non_championships.groupby("Continent")
# group_cont.count().head()

elite_non_championships.head()

,Date From,Date To,Name,Country,Category,Class,Season,Continent
0,24/10/2017,29/10/2017,Vuelta a Colombia Femenina Oro y Paz,COLOMBIA,WE,2.2,2018,Americas
1,23/10/2017,01/11/2017,Vuelta a Guatemala,GUATEMALA,ME,2.2,2018,Americas
2,04/11/2017,04/11/2017,Subaru Australian Open Criterium,AUSTRALIA,ME,CRT,2018,Oceania
3,04/11/2017,04/11/2017,Le Tour De France Saitama Criterium,JAPAN,ME,CRT,2018,Asia
4,04/11/2017,04/11/2017,Subaru Australian Open Criterium,AUSTRALIA,WE,CRT,2018,Oceania


In [30]:
#add in the coordinates for each race so we can geolocate them on a map later

uci_country_coord = pd.DataFrame()
uci_country_coord = pd.read_excel('uci_country_coord.xlsx', encoding='UTF-8')
uci_df = pd.merge(elite_non_championships, uci_country_coord, right_on='uci_name', left_on='Country', how="left")

df_nulls = uci_df[uci_df.isnull().any(axis=1)]
loc_nulls = df_nulls.groupby('Country')
loc_nulls['Country'].value_counts()

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.groupby.groupby.DataFrameGroupBy'>

In [34]:
#Look at the numbers of races per continent
grouped_df = base_df_cats.groupby(['Season','Category', 'Class'])
grouped_df.count()

Date From  Date To  Name  Country  Continent
Season Category Class                                              
2006   ME       1.1           98       98    98       98         98
                1.2           89       89    89       89         89
                1.HC          16       16    16       16         16
                2.1           43       43    43       43         43
                2.2           86       86    86       84         86
                2.HC          16       16    16       16         16
                CC             6        6     6        6          6
                CM             2        2     2        2          2
                CN           107      107   107      107        107
                JR             4        4     4        4          4
       WE       1.1            6        6     6        6          6
                1.2           13       13    13       13         13
                2.1           10       10    10       10         10
                2.2           14       14    14       14         14
                CC             7        7     7        7          7
                CDM           12       12    12       12         12
                CM             2        2     2        2          2
                CN            76       76    76       76         76
                JR             4        4     4        4          4
2007   ME       1.1           96       96    96       96         96
                1.2           93       93    93       93         93
                1.HC          17       17    17       17         17
                2.1           44       44    44       44         44
                2.2           96       96    96       96         96
                2.HC          16       16    16       16         16
                CC             8        8     8        8          8
                CM             4        4     4        4          4
                CN           101      101   101      101        101
       WE       1.1            8        8     8        8          8
                1.2           16       16    16       16         16
...                          ...      ...   ...      ...        ...
2018   ME       2.1           47       47    47       47         47
                2.2           78       78    78       78         78
                2.HC          21       21    21       21         21
                2.UWT         17       17    17       17         17
                CN             1        1     1        1          1
                CRT           64       64    64       64         64
       WE       1.1           21       21    21       21         21
                1.2           19       19    19       19         19
                1.WWT         16       16    16       16         16
                2.1            9        9     9        9          9
                2.2           14       14    14       14         14
                2.WWT          8        8     8        8          8
                CN             1        1     1        1          1
                CRT           25       25    25       25         25
2019   ME       1.1           69       69    69       69         69
                1.2           73       73    73       73         73
                1.HC          23       23    23       23         23
                1.UWT         21       21    21       21         21
                2.1           48       48    48       48         48
                2.2           64       64    64       64         64
                2.HC          21       21    21       21         21
                2.UWT         17       17    17       17         17
                CRT            6        6     6        6          6
       WE       1.1           23       23    23       23         23
                1.2           30       30    30       30         30
                1.WWT         16       16    16       16         16
           